# REST API for PACO server


The docs is available at http://localhost:8080/docs or at [docs](http://localhost:8080/docs) 

In [ ]:
import requests
import sys
sys.path.append('src')

headers = {
    "Content-Type": "application/json",
}
url = "http://127.0.0.1:8000/"

## Define the *BPMN+CPI*

In the following cell the *BPMN+CPI* is defined. 
 - expression: A string representing the BPMN expression, which defines the sequence and parallelism of tasks.
 - impacts: A dictionary where keys are task names and values are lists of impacts (e.g., costs, durations).
 - durations: A dictionary where keys are task names and values are lists representing the duration range [min, max] for each task.
 - probabilities: A dictionary where keys are natures (e.g., 'N1') and values are their probabilities.
 - loops_round: A dictionary for defining loop rounds, if any.
 - delays: A dictionary where keys are event names (e.g., 'C1') and values are their delays.
 - loops_probability: A dictionary for defining loop probabilities, if any.
 - impacts_names: A list of impact names (e.g., ['cost_electricity']).

### Example in Figure 8
#### Define and Check BPMN

In [ ]:
from src.utils.env import EXPRESSION, IMPACTS, DURATIONS, IMPACTS_NAMES, PROBABILITIES, DELAYS, LOOP_PROBABILITY, LOOP_ROUND, H
import graphviz
from IPython.display import display, SVG

bpmn = {
    EXPRESSION: "(Cutting, ((Bending, (HP^[N1]LP)) || (Milling, (FD/[C1]RD))), (HPHS / [C2] LPLS))",
    IMPACTS: {"Cutting": [10, 1], "Bending": [20, 1],
        "Milling": [50, 1], "HP": [5, 4], "LP": [8, 1],
        "FD": [30, 1], "RD": [10, 1], "HPHS": [40, 1],
        "LPLS": [20, 3]
    },
    DURATIONS: {"Cutting": [0, 1], "Bending": [0, 1],
        "Milling": [0, 1], "HP": [0, 2], "LP": [0, 1],
        "FD": [0, 1], "RD": [0, 1], "HPHS": [0, 1],
        "LPLS": [0, 2]},
    IMPACTS_NAMES: ["electric_energy", "worker hours"],
    PROBABILITIES: {"N1": 0.2},
    DELAYS: {"C1": 0, "C2": 0},
    LOOP_PROBABILITY : {}, LOOP_ROUND: {}, H: 0,
}

try:
    resp = requests.get(f'{url}create_bpmn', json={'bpmn': bpmn},  headers=headers)
    resp.raise_for_status()
    display(SVG(graphviz.Source(resp.json()['bpmn_dot']).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
    print(f"HTTP Error ({resp.status_code}):", resp.json())

In [ ]:
bpmn


# Create the parse tree and execution tree

In [ ]:
from paco.parser.parse_tree import ParseTree
from paco.execution_tree.execution_tree import ExecutionTree
from paco.explainer.bdd.bdds import bdds_from_json

parse_tree, execution_tree = None, None
try:
    resp = requests.get(f'{url}create_execution_tree', json={"bpmn": bpmn},  headers=headers)
    resp.raise_for_status()

    response = resp.json()
    parse_tree, _,_ = ParseTree.from_json(response["parse_tree"], len(bpmn["impacts_names"]), 0)

    bound = [135.0, 15.0]

    display(SVG(graphviz.Source(parse_tree.to_dot()).pipe(format="svg")))

    execution_tree = ExecutionTree.from_json(parse_tree, response["execution_tree"], bpmn["impacts_names"])
    dot = execution_tree.to_dot(state=True, executed_time=True, diff=True)
    display(SVG(graphviz.Source(dot).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
    print(f"HTTP Error ({resp.status_code}):", resp.json())

## Create Explained Strategy

In [ ]:
try:
    resp = requests.get(f'{url}create_strategy', json={"bpmn": bpmn, "bound": bound, "parse_tree": parse_tree.to_dict(), "execution_tree": execution_tree.to_dict()}, headers=headers)
    resp.raise_for_status()

    response = resp.json()
    print(response["result"])

    frontier_solution_id = set()
    if "frontier_solution" in response: # Solution Found
        frontier_solution_id = set(map(int, response["frontier_solution"].strip("[]").split(",")))

    # With frontier node in blue
    dot = execution_tree.to_dot(state=True, executed_time=False, diff=True, frontier=frontier_solution_id)
    display(SVG(graphviz.Source(dot).pipe(format="svg")))

    if "strategy_tree" in response:
        explained_choices = bdds_from_json(parse_tree, response["bdds"])
        print("1 is dashed line of BPMN or Parse Tree")
        for choice, bdd in explained_choices.items():
            print(f"{choice.name} : {bdd.typeStrategy}")
            svg_data = graphviz.Source(bdd.bdd_to_dot()).pipe(format="svg")
            display(SVG(svg_data))

        strategy_tree = ExecutionTree.from_json(parse_tree, response["strategy_tree"], bpmn["impacts_names"], explained_choices)

        #TREE_STATE
        dot = strategy_tree.to_dot(state=True, executed_time=False, diff=True)
        display(SVG(graphviz.Source(dot).pipe(format="svg")))

except requests.exceptions.HTTPError as e:
    print(f"HTTP Error ({resp.status_code}):", resp.json())

In [ ]:
# Check that dashed edges are present for Choice children with even index
resp = requests.get(f'{url}create_bpmn', json={'bpmn': bpmn}, headers=headers)
resp.raise_for_status()
dot = resp.json()['bpmn_dot']
print('style=dashed occurrences:', dot.count('style=dashed'))
print('sample dashed edges:')
for ln in [l for l in dot.splitlines() if 'style=dashed' in l][:8]:
    print('  ', ln)
